In [3]:
# import re
import sys
import logging
from datetime import datetime as dtm
from tqdm.notebook import tqdm
import pandas as pd

In [4]:
pd.set_option('display.max_colwidth', None)
pd.set_option('display.float_format', '{:.2f}'.format)
tqdm.pandas()

logging.basicConfig(
        format=u'[%(levelname)-8s] %(asctime)s | %(message)s',
        datefmt='%Y-%m-%d %H:%M:%S',
        level=logging.INFO,
        # level=logging.DEBUG,
        stream=sys.stdout,
    )

In [ ]:
# https://www.cian.ru/cat.php?deal_type=sale&engine_version=2&offer_type=flat&region=184723

# https://www.cian.ru/cat.php?deal_type=sale&offer_type=flat&region=184723

In [1]:
url='https://www.cian.ru/cat.php?deal_type=sale&offer_type=flat&region=184723'

In [2]:
from lib.downloader import DownloaderSimple

html = DownloaderSimple().get(url)

In [6]:
with open('tmp/cian.html','wt') as f: f.write(html)

In [10]:
import re
from datetime import datetime as dtm
from tqdm.notebook import tqdm
import pandas as pd
from bs4 import BeautifulSoup


In [ ]:
# <article data-name="CardComponent" 

In [ ]:
# <span data-mark="OfferTitle"
# <span data-mark="OfferSubtitle" 
# <span data-mark="Deadline"

# <a data-name="GeoLabel"

# <span data-mark="MainPrice"

# <p data-mark="PriceInfo" 

# <div data-name="Description" 

# <div data-name="LinkArea" class="_93444fe79c--container--kZeLu _93444fe79c--link--DqDOy">
# <a href="https://sevastopol.cian.ru/sale/flat/273181085/" class="_93444fe79c--link--eoxce">

# <div data-name="TimeLabel" 

In [36]:
def get_link(tag):
    return tag.find('a').attrs['href']

def get_ts(tag):
    return [ t.text for t in tag.find_all('span') ]

def get_adr(tag):
    return [ t.text for t in tag.find_all('a',{'data-name':'GeoLabel',}) ]

def parse_item(tag):
    return {
    'OfferTitle':tag.find('span',{'data-mark':'OfferTitle',}).text,
    
    #'OfferSubtitle': tag.find('span',{'data-mark':'OfferSubtitle',}).text,
    # 'Deadline': tag.find('span',{'data-mark':'Deadline',}).text,
    
    'GeoLabel':  get_adr(tag),  # tag.find('span',{'data-name':'GeoLabel',}).text,
        
    'MainPrice': tag.find('span',{'data-mark':'MainPrice',}).text,
    'PriceInfo': tag.find('p',{'data-mark':'PriceInfo',}).text,
    'Description': tag.find('div',{'data-name':'Description',}).text,
    'LinkArea': get_link( tag.find('div',{'data-name':'LinkArea',})), 
    'TimeLabel': get_ts( tag.find('div',{'data-name':'TimeLabel',})),  
    }
    
    

In [37]:
root = BeautifulSoup(html,'html.parser')
text = [
    parse_item(tag)
    for tag in root.find_all('article',{'data-name':'CardComponent',}) 
]


In [38]:
pd.DataFrame(text)

,OfferTitle,GeoLabel,MainPrice,PriceInfo,Description,LinkArea,TimeLabel
0,"2-комн. кв., 61,63 м², 7/10 этаж","[Севастополь, р-н Гагаринский, улица Военных Строителей, 7к2]",5 546 700 ₽,90 000 ₽/м²,"Прекрасная квартира с городской пропиской в новом доме у моря. \n5 минут до пляжа! Остановка в шаговой доступности , 15 минут до центра города!\n\nДвухконтурный газовый котел. Индивидуальное отопление, которое позволит Вам не только сократить коммунальные платежи, но и создать комфортную температуру в квартире. \nУстановлены приборы учета (газ, вода, электричество)\nВысокие потолки (2.8 м) и увеличенные стеклопакеты визуально делают квартиру еще более светлой и просторной.\n\nТерритория жилого комплекса полностью огорожена. \nУютный зеленый двор с удобными скамейками для отдыха и детская площадка с каруселями и горками, которая обязательно понравится Вашим детям!\n\nМоре и пляж, магазины и социальные объекты (новая школа и детский сад), исторические и культурные объекты Севастополя все это рядом с домом! \n\nВысокая степень готовности дома. Сдача дома 4 квартал 2022 год\nРассрочка от застройщика. Материнский капитал. \n\nПрекрасное вложение как для проживания, так и для сдачи в аренду\nЗвоните! Продам квартиру в Севастополе.",https://sevastopol.cian.ru/sale/flat/273181007/,"[10 часов назад, сегодня, 10:07]"
1,"1-комн. апарт., 16,6 м², 1/3 этаж","[Севастополь, р-н Гагаринский, Казачья Бухта мкр]",1 494 000 ₽,90 000 ₽/м²,"Дом ""Авангард"" отличается уникальной архитектурой и планировками.\nВ доме 2 трехэтажные секции. Дом построен по монолитно-каркасной технологии.\nЖК ""Скифия"" расположен в одном из самых перспективных и динамично развивающихся районов города в курортной части бухты Казачья в Гагаринском районе Севастополя. Благодаря сочетанию качества, экологичности и доступности покупка недвижимости в ЖК ""Скифия"" идеальный вариант для жизни и инвестиций.\n\nРассрочка от застройщика. Материнский капитал.\n\n+ О ЖК: Проект строительства жилого комплекса предусматривает полное благоустройство внутренней территории, обустроенные детские площадки и места для отдыха. Активное озеленение хвойными деревьями.\n\n+ Инфраструктура: Перспективный для жизни и инвестиций динамично развивающийся район. Новая школа, детский сад, рынок. Недалеко от комплекса осуществляется строительство нового культурно-досугового центра.\n\n+ О квартире: Высота потолков 2,8 м, вывод канализации, ХВС, установлены приборы учета. Море в шаговой доступности.\n\n+ Транспортная доступность: Менее 5 минут до остановки общественного транспорта, 15 минут до центра города на автомобиле. Продается квартира в Севастополе.",https://sevastopol.cian.ru/sale/flat/273181085/,"[вчера, вчера, 10:04]"
2,"2-комн. кв., 55,2 м², 4/10 этаж","[Севастополь, р-н Гагаринский, мкр. Камышовая бухта, улица Павла Корчагина, 23]",10 692 000 ₽,193 696 ₽/м²,"Продаётся 2-комнатная квартира в сданном доме (Дом 1), срок сдачи: II-кв. 2022, общей площадью 55.2 кв.м., на 4 этаже. Новый жилой комплекс ""Апельсин"" расположен на улице Корчагина в 10 минутах от моря и самого оборудованного пляжа Севастополя. Дизайн дома задуман в едином стиле с соседними объектами, но обладает своими уникальными особенностями. \nАРХИТЕКТУРНЫЙ СТИЛЬ \nРасцветка дома выполнена в светлых южных оттенках в тон соседним ЖК ""Каравелла"" и ЖК ""Дом на Корчагина"". Благодаря этому жилые комплексы превратятся в полноценный микрорайон, возведенный в единой концепции. \nБЕЗУПРЕЧНАЯ ИНФРАСТРУКТУРА \nВ районе жилого комплекса располагаются удобные транспортные развязки, остановки общественного транспорта, магазины, банки, аптеки. Рядом находятся социальные объекты - школа и детский сад. \nМЕСТА ДЛЯ РАЗВЛЕЧЕНИЙ \nВозле жилого комплекса расположен любимый у детей ""Динопарк"" с площадками для активного отдыха. Взрослые смогут посещать спектакли и концерты в Культурно-информационном центре, куда регулярно приезжают с гастролями российские звезды. В пяти минутах ходьбы от объекта находится крупный торговый центр ""Апельсин"" со множеств

In [15]:
# text[0]

In [14]:
# text[1]